In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial



In [8]:
data = pd.read_csv('data.csv',header=None)
labels = pd.read_csv('label.csv',names=['label'],header=None)

In [10]:
labels.shape

(10000, 1)

In [11]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [12]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0

            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [13]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [14]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [15]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [16]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  24214.56341463415
Tolerance for the Iteration  1 :  4763.257874260815
Tolerance for the Iteration  2 :  2509.0703703703703
Tolerance for the Iteration  3 :  2362.1920058211854
Tolerance for the Iteration  4 :  2301.3604302749736
Tolerance for the Iteration  5 :  2918.787544367864
Tolerance for the Iteration  6 :  2801.867152455232
Tolerance for the Iteration  7 :  2158.803415347659
Tolerance for the Iteration  8 :  1284.4762349864732
Tolerance for the Iteration  9 :  1001.2348165715193
Tolerance for the Iteration  10 :  844.4600780649359
Tolerance for the Iteration  11 :  583.1973315528203
Tolerance for the Iteration  12 :  462.0270072790804
Tolerance for the Iteration  13 :  257.77052871492793
Tolerance for the Iteration  14 :  221.85688946889547
Tolerance for the Iteration  15 :  189.219936314551
Tolerance for the Iteration  16 :  208.69735133463877
Tolerance for the Iteration  17 :  129.64466081782382
Tolerance for the Iteration  18 :  88.7393920822

In [17]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [18]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25454304340.476444


In [19]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([2, 4, 0, 7, 3, 4, 5, 6, 1, 8])

In [20]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.645

In [21]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  36315.06580366775
Tolerance for the Iteration  1 :  10332.198479152887
Tolerance for the Iteration  2 :  9107.486706218413
Tolerance for the Iteration  3 :  9924.950532552659
Tolerance for the Iteration  4 :  11852.857133333333
Tolerance for the Iteration  5 :  3054.852996204934
Tolerance for the Iteration  6 :  1220.5438472775566
Tolerance for the Iteration  7 :  936.9166565262077
Tolerance for the Iteration  8 :  965.0841952191233
Tolerance for the Iteration  9 :  1482.9759400215748
Tolerance for the Iteration  10 :  1035.716604835924
Tolerance for the Iteration  11 :  368.3271873965442
Tolerance for the Iteration  12 :  324.10659470595925
Tolerance for the Iteration  13 :  0.0


In [22]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34364573698.62264


In [23]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 0, 0, 3, 7, 4, 5, 7, 3, 8])

In [24]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.215

In [25]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  30447.858235294116
Tolerance for the Iteration  1 :  8563.813767929267
Tolerance for the Iteration  2 :  3053.729866962306
Tolerance for the Iteration  3 :  1497.5509278801605
Tolerance for the Iteration  4 :  826.453798270031
Tolerance for the Iteration  5 :  630.2447496434121
Tolerance for the Iteration  6 :  705.1632621272804
Tolerance for the Iteration  7 :  784.2842579750347
Tolerance for the Iteration  8 :  599.9003316534498
Tolerance for the Iteration  9 :  589.0188097745022
Tolerance for the Iteration  10 :  479.0917602374152
Tolerance for the Iteration  11 :  484.5633739766883
Tolerance for the Iteration  12 :  459.4042003805322
Tolerance for the Iteration  13 :  370.5925043401301
Tolerance for the Iteration  14 :  199.13709677419382
Tolerance for the Iteration  15 :  151.43453968253968
Tolerance for the Iteration  16 :  155.51282636745916
Tolerance for the Iteration  17 :  109.72090729783035
Tolerance for the Iteration  18 :  96.2866250446505

In [26]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [27]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25454304340.476444
Jacard SSE: 34364573698.62264
Cosine SSE : 25430375772.008137


In [28]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([1, 6, 2, 1, 8, 7, 5, 0, 3, 4])

In [29]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.63

In [30]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.645
Jacard accuracy: 0.215
Cosine accuracy : 0.63


In [33]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict, data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a, b in zip(centroid_value_dict[i], dp):
                    sse_cluster += (a - b) ** 2
            sse_data += sse_cluster
        return sse_data
    
    def Initialize_Centroids(self, data, K):
        m = data.shape[0]
        centroid_value_dict = {}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self, centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self, data, K, max_iter=20, mode=1, tol=10, stopping_condition='centroid_change'):
        # Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        # Mode = 2 => Jaccard
        # Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data, K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        sse_list = []
        convergence = False
        
        while (count < max_iter) and not convergence:
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i] = []
                
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode == 1:
                    distance_measure = [np.linalg.norm(x - centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode == 2:
                    distance_measure = [self.jaccard_similarity(list(x), centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode == 3:
                    distance_measure = [1 - scipy.spatial.distance.cosine(x, list(centroid_value_dict[j])) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids = dict(centroid_value_dict)
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid], axis=0)
            
            current_tol = -1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point - prev_centroid_point))
                current_tol = max(change, current_tol)
                
            sse = self.calculate_SSE(centroid_value_dict, centroid_dict, data)
            sse_list.append(sse)
            
            if stopping_condition == 'centroid_change' and current_tol < tol:
                convergence = True
                break
            
            if stopping_condition == 'sse_increase' and len(sse_list) > 1 and sse_list[-1] > sse_list[-2]:
                convergence = True
                break
                
            count += 1
            if stopping_condition == 'max_iter' and count == max_iter:
                convergence = True
                break

                
        return centroid_value_dict, centroid_dict, sse_list


In [ ]:
model = KMeans()

# Train Euclidean-K-means
centroids_euc, clusters_euc, sse_list_euc = model.train_Kmeans(data, K=10, max_iter=100, mode=1, tol=10, stopping_condition='centroid_change')

# Train Jaccard-K-means
centroids_jac, clusters_jac, sse_list_jac = model.train_Kmeans(data, K=10, max_iter=100, mode=2, tol=10, stopping_condition='centroid_change')

# Train Cosine-K-means
centroids_cos, clusters_cos, sse_list_cos = model.train_Kmeans(data, K=10, max_iter=100, mode=3, tol=10, stopping_condition='centroid_change')
